### Intall libraries

In [35]:
from IPython.display import clear_output

!pip install datasets
!pip install scikit-learn
!pip install RapidFuzz
clear_output(wait=False)

### Evaluate

Dataset chosen for evaluation: [vishnun/SpellGram](https://huggingface.co/datasets/vishnun/SpellGram)

Load eval dataset

In [40]:
from datasets import load_dataset

ds = load_dataset("vishnun/SpellGram", trust_remote_code=True)
ds = ds["train"].shuffle(seed=42).select(range(10000))

In [41]:
ds

Dataset({
    features: ['source', 'target'],
    num_rows: 10000
})

In [42]:
ds[0]

{'source': 'i havv arranged a conference call to oiscuss these matters further',
 'target': 'i have arranged a conference call to discuss these matters further'}

Load spelling corrector

In [43]:
from spelling_corrector_baseline import ContextSensitiveSpellCorrector


ngram_model_path = r"models\trigram_model_shakespeare.pkl"
spell_corrector = ContextSensitiveSpellCorrector(ngram_model_path, n=3)

sentence = "to be or not to ve"
corrected_sentence = spell_corrector.correct_sentence(sentence)
print("  initial: ", sentence)
print("corrected: ", corrected_sentence)

  initial:  to be or not to ve
corrected:  to be or not to be


Evaluation of baseline model

In [44]:
from rapidfuzz.distance import Levenshtein
from tqdm import tqdm


def evaluate_spelling_corrector(corrector, test_data):
    total_words = 0
    correct_words = 0
    total_chars = 0
    total_edit_distance = 0
    
    for example in tqdm(test_data):
        source = example["source"]
        target = example["target"]
        corrected = corrector.correct_sentence(source)
        
        # Word Accuracy
        target_words = target.split()
        corrected_words = corrected.split()
        
        total_words += len(target_words)
        correct_words += sum(1 for tw, cw in zip(target_words, corrected_words) if tw == cw)
        
        # Character Error Rate (CER)
        total_chars += len(target)
        total_edit_distance += Levenshtein.distance(target, corrected)

    word_accuracy = correct_words / total_words if total_words > 0 else 0
    cer = total_edit_distance / total_chars if total_chars > 0 else 0

    return {"word_accuracy": word_accuracy, "character_error_rate": cer}

In [46]:
result = evaluate_spelling_corrector(spell_corrector, ds)

print(f"Word accuracy: {result['word_accuracy']:.4f}")
print(f"Character error rate: {result['character_error_rate']:.4f}")

100%|██████████| 10000/10000 [29:41<00:00,  5.61it/s]

Word accuracy: 0.7058
Character error rate: 0.0781


Evaluation improved model

In [ ]:
# TO_DO